<a href="https://colab.research.google.com/github/moh2236945/Market-Profile/blob/master/quant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install backtrader
!pip install backtrader[plotting]
!pip3 install alpaca-backtrader-api

     |████████████████████████████████| 419kB 5.3MB/s 
     |████████████████████████████████| 102kB 5.1MB/s 
     |████████████████████████████████| 81kB 5.6MB/s 
     |████████████████████████████████| 204kB 10.7MB/s 
  Created wheel for trading-calendars: filename=trading_calendars-1.11.11-cp36-none-any.whl size=134435 sha256=e013d1b06694882ab1d19c4dc524ff6c54133846cbf85a6eda3684aabe6e10ca
  Stored in directory: /root/.cache/pip/wheels/3b/f7/ae/d1649f3ad0c85e63ffda3fb2df782b822ef02a67c5f1129e2a
Successfully built trading-calendars


In [9]:
import alpaca_backtrader_api as tradeapi
import backtrader as bt
from datetime import datetime

api_key = "PK6AITFDR90TE94MP4AA"
api_secret = "mWrFJCneXay/LviBdYf7grsJoX8QelASVgKMz6/N"
base_url = 'https://paper-api.alpaca.markets'
import alpaca_trade_api as tradeapi

api = tradeapi.REST(api_key, api_secret, base_url='https://paper-api.alpaca.markets') # or use ENV Vars shown below
account = api.get_account()
if account.trading_blocked:
    print('Account is currently restricted from trading.')

# Check how much money we can use to open new positions.
print('${} is available as buying power.'.format(account.buying_power))

# Check our current balance vs. our balance at the last market close
balance_change = float(account.equity) - float(account.last_equity)
print(f'Today\'s portfolio balance change: ${balance_change}')
# Get a list of all active assets.
active_assets = api.list_assets(status='active')

# Filter the assets down to just those on NASDAQ.
nasdaq_assets = [a for a in active_assets if a.exchange == 'NASDAQ']
#print(nasdaq_assets)
# Check if AAPL is tradable on the Alpaca platform.
aapl_asset = api.get_asset('AAPL')
if aapl_asset.tradable:
    print('We can trade AAPL.')

# Check if the market is open now.
clock = api.get_clock()
print('The market is {}'.format('open.' if clock.is_open else 'closed.'))

# Check when the market was open on Dec. 1, 2018
date = '2018-12-01'
calendar = api.get_calendar(start=date, end=date)[0]
print('The market opened at {} and closed at {} on {}.'.format(
    calendar.open,
    calendar.close,
    date
))
# Get daily price data for AAPL over the last 5 trading days.
barset = api.get_barset('AAPL', 'day', limit=5)
aapl_bars = barset['AAPL']

# See how much AAPL moved in that timeframe.
week_open = aapl_bars[0].o
week_close = aapl_bars[-1].c
percent_change = (week_close - week_open) / week_open * 100
print('AAPL moved {}% over the last 5 days'.format(percent_change))

$400000 is available as buying power.
Today's portfolio balance change: $0.0
We can trade AAPL.
The market is closed.
The market opened at 09:30:00 and closed at 16:00:00 on 2018-12-01.


In [55]:
import alpaca_backtrader_api as alpaca
import backtrader as bt
import pytz
from datetime import datetime
#from local_settings import alpaca_paper

ALPACA_KEY_ID = 'PK6AITFDR90TE94MP4AA'
ALPACA_SECRET_KEY = 'mWrFJCneXay/LviBdYf7grsJoX8QelASVgKMz6/N'
ALPACA_PAPER = True

fromdate = datetime(2020,8,5)
todate = datetime(2020,8,10)

tickers = ['SPY']
timeframes = {
    '1Min':1,
    '30Min':30,
    '1H':60,
}

class RSIStack(bt.Strategy):

    def next(self):
        for i in range(0,len(self.datas)):
            print(f'{self.datas[i].datetime.datetime(ago=0)} \
            {self.datas[i].p.dataname}: OHLC: \
                  o:{self.datas[i].open[0]} \
                  h:{self.datas[i].high[0]} \
                  l:{self.datas[i].low[0]} \
                  c:{self.datas[i].close[0]} \
                  v:{self.datas[i].volume[0]}' )

cerebro = bt.Cerebro()
cerebro.addstrategy(RSIStack)
cerebro.broker.setcash(100000)
cerebro.broker.setcommission(commission=0.0)

store = alpaca.AlpacaStore(
    key_id=ALPACA_KEY_ID,
    secret_key=ALPACA_SECRET_KEY,
    paper=ALPACA_PAPER
)

if not ALPACA_PAPER:
    print(f"LIVE TRADING")
    broker = store.getbroker()
    cerebro.setbroker(broker)

DataFactory = store.getdata

for ticker in tickers:
    for timeframe, minutes in timeframes.items():
        print(f'Adding ticker {ticker} using {timeframe} timeframe at {minutes} minutes.')

        d = DataFactory(
            dataname=ticker,
            timeframe=bt.TimeFrame.Minutes,
            compression=minutes,
            fromdate=fromdate,
            todate=todate,
            historical=True)

        cerebro.adddata(d)

cerebro.run()
print("Final Portfolio Value: %.2f" % cerebro.broker.getvalue())
cerebro.plot(style='candlestick', barup='green', bardown='red')


11-Sep-20 03:17:08 DEBUG     Resetting dropped connection: paper-api.alpaca.markets


Adding ticker SPY using 1Min timeframe at 1 minutes.
Adding ticker SPY using 30Min timeframe at 30 minutes.
Adding ticker SPY using 1H timeframe at 60 minutes.


11-Sep-20 03:17:09 DEBUG     https://paper-api.alpaca.markets:443 "GET /v2/assets/SPY HTTP/1.1" 200 226
11-Sep-20 03:17:09 DEBUG     Resetting dropped connection: data.alpaca.markets
11-Sep-20 03:17:09 DEBUG     https://paper-api.alpaca.markets:443 "GET /v2/assets/SPY HTTP/1.1" 200 226
11-Sep-20 03:17:09 DEBUG     Resetting dropped connection: data.alpaca.markets
11-Sep-20 03:17:09 DEBUG     https://paper-api.alpaca.markets:443 "GET /v2/assets/SPY HTTP/1.1" 200 226
11-Sep-20 03:17:09 DEBUG     Resetting dropped connection: data.alpaca.markets
11-Sep-20 03:17:09 DEBUG     https://data.alpaca.markets:443 "GET /v1/bars/minute?symbols=SPY&limit=1000&end=2020-08-07T15%3A59%3A00-04%3A00 HTTP/1.1" 200 None
11-Sep-20 03:17:09 DEBUG     https://data.alpaca.markets:443 "GET /v1/bars/minute?symbols=SPY&limit=1000&end=2020-08-07T15%3A59%3A00-04%3A00 HTTP/1.1" 200 None
11-Sep-20 03:17:09 DEBUG     https://data.alpaca.markets:443 "GET /v1/bars/minute?symbols=SPY&limit=1000&end=2020-08-07T15%3A59%3A0

2020-08-05 14:00:00             SPY: OHLC:                   o:331.99                   h:332.06                   l:331.85                   c:331.92                   v:31756.0
2020-08-05 14:00:00             SPY: OHLC:                   o:331.99                   h:332.12                   l:331.75                   c:331.99                   v:1063104.0
2020-08-05 14:00:00             SPY: OHLC:                   o:331.99                   h:332.16                   l:331.63                   c:331.85                   v:1796076.0
2020-08-05 14:01:00             SPY: OHLC:                   o:331.91                   h:331.93                   l:331.75                   c:331.8                   v:10946.0
2020-08-05 14:00:00             SPY: OHLC:                   o:331.99                   h:332.12                   l:331.75                   c:331.99                   v:1063104.0
2020-08-05 14:00:00             SPY: OHLC:                   o:331.99                   h:332.16    

[[<Figure size 432x288 with 8 Axes>]]